# Opioid Treatment Center Matching Tool 
## Team 2
## Parde RAND Tech Lab Hackathon
## Tool Creators: Simon Hollands, Luke Irwin, John Speed Meyers, Ben Smith
## Larger team includes: Erin Duffy, Sangita Baxi, and Juliana Chen Peraza
## 11/10/2018

## Description: This tool helps match those with opioid addiction with treatment centers that are nearby and meet their personal preferences. The tool places the 25 nearest centers on a map and allows users to filter a table of these treatment centers with three buttons. 

## Technical notes: This is written in Python. Simon used Python 2. John Speed used Python 3. Beware. Many packages are required, but especially Bokeh, pandas, numpy, and pyproj. Do not use the RAND staff internet connection to download these. Switch to r.internet to download these. The download does not work on the RAND staff internet. Also, for Python 3.6 and 3.7 users, you might need to download Visual Basic 14.

In [1]:
## These are the variables of interest. 
## TO DO: Ben, can you please select some variables from your 2018 dataset that you think are especially
## useful for a potential patient selecting a treatment center?
## TO DO: Add a description of the variables we select.
vars_of_interest= ['ACC', 'ACM', 'ACRES', 'ACU' ,'AD', 'ADAMS', 'ADD', 'ADLT', 'ADM','ADTX']
vars_of_interest_str = str(vars_of_interest).strip('[]')

In [2]:
## This block contains functions that aid in calculating the treatment centers closed to the long-lat coordinates
## of the centroid of a zipcode that the user specifies.

import pandas as pd
from math import sin, cos, sqrt, atan2, radians, asin ## Required match functions

#making an edit
def apply_radian_lat(row):
    ##I NEED A DESCRIPTION##
    return radians(row['lat'])

def apply_radian_long(row):
    ##I NEED A DESCRIPTION##
    return radians(row['long'])

def create_distance_lat(row):
    ## Find distance between latitudes of two coordinates##    
    return (row['lat2']-row['lat1'])

def create_distance_long(row):
    ## Find distance between longitude of two coordinates##   
    return (row['long2']-row['lon1'])

def create_distance(row):
    ##Find total distance between two coordinates##
    a=sin(row['dlat'] / 2)**2 + cos(row['lat1']) * cos(row['lat2']) * sin(row['dlon'] / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return 6373.0 * c

def algo(user_long, user_lat, df2):
    ## Find total distance between latitude and longitude associated with user-specified zip code
    ## and the 25 nearest treatment centers
    #df2 = df[df[var1] == 1]
    df2['lat1']=radians(user_lat) ## Get user latitude
    df2['lon1']=radians(user_long) ## Get user longitude
    df2['lat2'] = df2.apply(apply_radian_lat, axis=1)
    df2['long2']= df2.apply(apply_radian_long, axis=1)
    df2['dlon'] = df2.apply(create_distance_long, axis=1)
    df2['dlat']=df2.apply(create_distance_lat, axis=1)
    df2['distance']=df2.apply(create_distance, axis=1)
    df2=df2.sort_values(by=['distance'],ascending=True).reset_index()
    out=df2.head(50)
    return out


In [3]:
## This block produces the actual Bokeh tool. It produces a dashboard.

import numpy as np
import pandas as pd

from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import widgetbox
from bokeh.models import ColumnDataSource, CustomJS, Div, Filter
from bokeh.models.widgets import DataTable,TableColumn, Button, Select, Paragraph
from bokeh.plotting import figure
from bokeh import events
from bokeh.layouts import column, row
from bokeh.tile_providers import CARTODBPOSITRON ## Necessary for mapping
from bokeh.models import HoverTool ## For being able to click on individual centers
from pyproj import Proj, transform ## Necessary for coordinate transformation

output_notebook() ## Keep dashboard within the jupyter notebook, change command output_file to create standalone HTML

## Read in the treatment center data
df = pd.read_csv(r"C:\Users\jmeyers\Desktop\opioid_dashboard-master\opioid_dashboard-master\clean_data\treatmentCenters2018_cleaned.csv")

## Read in a dataset of zipcodes and the longitude and latitudes associated with their centroid
zipcode = pd.read_csv(r"C:\Users\jmeyers\Desktop\opioid_dashboard-master\opioid_dashboard-master\data\zipcodes_to_longlat.csv")

## Ask for user input - Beware input command - Different between Python 2 and Python 3
user_zip = float(input("What is your zip code? "))

## Extract long and lat from user zipcode using the zipcode-longlat dataset
user_long = zipcode[zipcode['ZIP'] == user_zip]['LNG']
user_lat = zipcode[zipcode['ZIP'] == user_zip]['LAT']

## Question: What if the user only wants to choose one or two, or more than three?
## User chooses 3 variables that they want to filter on. See above for creation of vars_of_interest_str 
## TO DO: MAKE IT SO THAT THE USER CAN SELECT AS MANY FILTERS AS HE OR SHE WANTS TO
## TO DO: GIVE THE FILTERS MEANINGFUL NAMES AND NOT JUST ACRONYMS
user_filter =  input("Enter 3 Filter from this list, separated by spaces" + vars_of_interest_str )
user_filter_list = user_filter.split() #Put the user input into a list

#Check if entered correctly
for element in user_filter_list:
    if element not in vars_of_interest:
        raise ValueError('Please enter correct filter variable')

#Grab subset of data closest to entered long/lat
data = algo(user_long,user_lat, df)
data['Matching_Accuracy']=1 #Initialize this var to be used later

# Update document every time this script is run
def modify_doc(doc):
    
    ## Initialize data source
    source = ColumnDataSource(data)
    
    ## Set columns to be displayed -- field is the actual name of the variable to be selected and title
    ## is the name you wish to display in the table GUI
    ## TO DO: ADD ADDRESS AND PHONE NUMBER
    ## TO DO: DO HOVER TOOL AND BE ABLE TO HOVER OVER DOT AND SEE PHONE NUMBER
    columns = [
        TableColumn(field="hash", title="Hash"),
        TableColumn(field="long", title="Longitude"),
        TableColumn(field="lat", title="Latitude"),
        TableColumn(field=user_filter_list[0], title=user_filter_list[0]), #These are for the user chosen buttons
        TableColumn(field=user_filter_list[1], title=user_filter_list[1]),
        TableColumn(field=user_filter_list[2], title=user_filter_list[2]),
        TableColumn(field="Matching_Accuracy", title="Matching_Accuracy"),      
    ]
    
    ############################
    ## CREATE DATA TABLE ELEMENT
    ############################
    
    table = DataTable(
        source=source,
        columns=columns,
        width=800, 
        height=300
    )
    
    #####################
    ## CREATE MAP ELEMENT
    #####################
    
    ## Create base map range - this is a web mercator map
    ## TO DO: HAVE RANGE SELECT ONLY THE ZIP CODE AREA THAT THE USER SPECIFIES
    ## ONE POTENTIAL SOLUTION IS TO FIND THE WESTERNMOST, EASTERNMOST, SOUTHERNMOST, NORTHERNMOST TREATMENT CENTER
    ## COORDINATES AND THEN RESIZE THE RANGES APPROPRIATELY
    treatment_map = figure(x_range=(-12000000, -10000000), y_range=(1200000, 7500000),
                        x_axis_type="mercator", y_axis_type="mercator")

    ## Convert long lat to web mercator coordinates with pyproj module
    ## TO DO: DO MAP COORDINATE TRANSFORMATION IN UNDERLYING DATASET SO THAT THIS CALCULATION IS A) UNNECESSARY
    ## AND B) SO THAT TOOLTIPS HOVER TOOL CAN ACCESS THE DATASET AND NOT JUST THE LONG AND LAT COLUMNS
    coord = transform(Proj(init='epsg:4326'), Proj(init='epsg:3857'), source.data['long'], source.data['lat'])
    
    treatment_map.add_tile(CARTODBPOSITRON) ## Add base layer map
    
    ## Add treatment centers to map
    treatment_map.circle(x=coord[0], y=coord[1], size=4, fill_color="blue", fill_alpha=0.8)
    
    ######################
    ## FILTERING FUNCTIONS
    ######################
    
    ## These functions filter the table at the bottom of the dashboard
    ## TO DO: These filters can only be applied one at a time. This was not our original intention. If there
    ## is sufficient time, we would make it so that multiple filters can be applied at one.
    
    ## Reset table so that the filters are effectively turned off
    def reset_table():
        data3 = data
        new2 = ColumnDataSource(data3)
        source.data = new2.data
        
    ## Generate first button functionality 
    ## Filter table based on first variable specified
    ## Select only treatment centers where variable equals 1
    def update1():
        #Do the filtering
        data2 = data[data[user_filter_list[0]]==1]
        #Random matching accuracy variable to look fancy
        data2['Matching_Accuracy'] = np.random.randint(90, 100, data2.shape[0])    
        data2['Matching_Accuracy'] = data2.Matching_Accuracy.map(str) + " % "         
        new = ColumnDataSource(data2)
        source.data = new.data

    ## Generate second button functionality 
    ## Filter table based on first variable specified
    ## Select only treatment centers where variable equals 1
    def update2():
        #Do the filtering
        data2 = data[data[user_filter_list[1]]==1]
        #Random matching accuracy variable to look fancy
        data2['Matching_Accuracy'] = np.random.randint(90, 100, data2.shape[0])    
        data2['Matching_Accuracy'] = data2.Matching_Accuracy.map(str) + " % " 
        new = ColumnDataSource(data2)
        source.data = new.data
        
    ## Generate third button functionality 
    ## Filter table based on first variable specified
    ## Select only treatment centers where variable equals 1
    def update3():
        #Do the filtering
        data2 = data[data[user_filter_list[2]]==1]
        #Random matching accuracy variable to look fancy
        data2['Matching_Accuracy'] = np.random.randint(90, 100, data2.shape[0])    
        data2['Matching_Accuracy'] = data2.Matching_Accuracy.map(str) + " % " 
        new = ColumnDataSource(data2)
        source.data = new.data

    ## Create four buttons -- one button for each of three attributes and a reset button
    ## Each button's name reflects the variable it filters
    bt = Button(label="Select " + user_filter_list[0], button_type="success", width=200)
    bt1 = Button(label="Select " + user_filter_list[1], button_type="success", width=200)
    bt2 = Button(label="Select " + user_filter_list[2], button_type="success", width=200)
    bt_reset = Button(label="Reset All", button_type="success", width=50)
    
    ## Associate button with functionality
    bt.on_click(update1)
    bt1.on_click(update2)
    bt2.on_click(update3)
    bt_reset.on_click(reset_table)
    
    ## Set up layout of dashboard
    button_row1 = row(bt, bt_reset, sizing_mode='fixed')
    layout = column(treatment_map, button_row1, bt1, bt2, table) 
    
    doc.add_root(layout)

show(modify_doc) # Display table and button

Loading BokehJS ...

What is your zip code? 90025
Enter 3 Filter from this list, separated by spaces'ACC', 'ACM', 'ACRES', 'ACU', 'AD', 'ADAMS', 'ADD', 'ADLT', 'ADM', 'ADTX'ACC ACM AD


C:\Users\jmeyers\AppData\Local\Continuum\anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:143: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\jmeyers\AppData\Local\Continuum\anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:144: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
ERROR:bokeh.server.protocol_handler:error handling message Message 'PATCH-DOC' (revision 1): DeserializationError("Seq(Any) expected a list or None, got {'0': 1, '1': 1, '2': 1, '3': 1, '4': 1}",)
C:\Users\jmeyers\AppData\Local\Continuum\anaconda